In [ ]:
from django.db import models
from django.db.models.sql.query import Query
from django.db.models.sql.datastructures import Join
from django.db.models.query import ModelIterable
from django.contrib.contenttypes.models import ContentType
from django.contrib.contenttypes.fields import GenericRelation


class IncludeModelIterable(ModelIterable):
    
    def __iter__(self):
        print(self.queryset._includes)
        for instance in super(IncludeModelIterable, self).__iter__():
            for includer in self.queryset._includes.values():
                includer.extract(instance)
                
            yield instance
            
            
class AbstractIncluder(object):
    def __init__(self, alias, field):
        self.alias = alias
        self.field = field
        self.model = field.related_model
        self.host_model = field.model
        

class ManyToOneIncluder(AbstractIncluder):
    
    def extract(self, instance):
        data = getattr(instance, self.alias)
        delattr(instance, self.alias)
        setattr(instance, self.field.get_cache_name(), self.model.from_db(instance._state.db, None, data))
        
    def extra_query(self):
        return {self.alias:'''
        SELECT json_build_array({select})
        FROM "{table}" AS "{alias}"
        WHERE "{alias}"."{join_column}" = "{host}"."{host_join_column}"
        '''.format(
            alias=self.alias,
            table=self.model._meta.db_table,
            join_column=self.field.target_field.column,
            host=self.host_model._meta.db_table,
            host_join_column=self.field.column,
            select=', '.join('"{}"."{}"'.format(self.alias, f.column) for f in self.model._meta.concrete_fields)
        )}


class OneToManyIncluder(AbstractIncluder):
    
    def extract(self, instance):
        data = getattr(instance, self.alias)
        delattr(instance, self.alias)
        
        if not hasattr(instance, '_prefetched_objects_cache'):
            instance._prefetched_objects_cache = {}
        instance._prefetched_objects_cache[self.field.name] = self.model.objects.none()
        instance._prefetched_objects_cache[self.field.name]._result_cache = [
            self.model.from_db(instance._state.db, None, datum)
            for datum in data
        ]
    
    def extra_query(self):
        host_join_column, join_column = self.field.get_joining_columns()[0]
        return {self.alias:'''
        SELECT json_agg(json_build_array({select}))
        FROM "{table}" AS "{alias}"
        WHERE "{alias}"."{join_column}" = "{host}"."{host_join_column}"
        '''.format(
            alias=self.alias,
            table=self.model._meta.db_table,
            join_column=join_column,
            host=self.host_model._meta.db_table,
            host_join_column=host_join_column,
            select=', '.join('"{}"."{}"'.format(self.alias, f.column) for f in self.model._meta.concrete_fields)
        )}
    
    
class OneToManyIncluder(AbstractIncluder):
    
    def extract(self, instance):
        data = getattr(instance, self.alias)
        delattr(instance, self.alias)
        
        if not hasattr(instance, '_prefetched_objects_cache'):
            instance._prefetched_objects_cache = {}
        instance._prefetched_objects_cache[self.field.name] = self.model.objects.none()
        instance._prefetched_objects_cache[self.field.name]._result_cache = [
            self.model.from_db(instance._state.db, None, datum)
            for datum in data
        ]
    
    def extra_query(self):
        host_join_column, join_column = self.field.get_joining_columns()[0]
        return {self.alias:'''
        SELECT json_agg(json_build_array({select}))
        FROM "{table}" AS "{alias}"
        WHERE "{alias}"."{join_column}" = "{host}"."{host_join_column}"
        '''.format(
            alias=self.alias,
            table=self.model._meta.db_table,
            join_column=join_column,
            host=self.host_model._meta.db_table,
            host_join_column=host_join_column,
            select=', '.join('"{}"."{}"'.format(self.alias, f.column) for f in self.model._meta.concrete_fields)
        )}

    
class GenericRelationIncluder(OneToManyIncluder):
    def extra_query(self):
        join_column, host_join_column = self.field.get_joining_columns()[0]
        return {self.alias:'''
        SELECT json_agg(json_build_array({select}))
        FROM "{table}" AS "{alias}"
        WHERE "{alias}"."{join_column}" = "{host}"."{host_join_column}"
        AND "{alias}"."{content_type}" = {content_type_id}
        '''.format(
            alias=self.alias,
            table=self.model._meta.db_table,
            join_column=join_column,
            host=self.host_model._meta.db_table,
            host_join_column=host_join_column,
            select=', '.join('"{}"."{}"'.format(self.alias, f.column) for f in self.model._meta.concrete_fields),
            content_type=self.model._meta.get_field(self.field.content_type_field_name).column,
            content_type_id=ContentType.objects.get_for_model(self.host_model).pk
        )}


class IncludeQuerySet(models.QuerySet):

    def __init__(self, *args, **kwargs):
        super(IncludeQuerySet, self).__init__(*args, **kwargs)
        self._includes = {}
        self._iterable_class = IncludeModelIterable

    def include(self, *related_names):
        clone = self._clone()
        for related_name in related_names:
            field = self.model._meta.get_field(related_name)
            if isinstance(field, GenericRelation):
                clone._includes[field] = GenericRelationIncluder('__garbage__' + field.name, field)
            elif field.many_to_one:
                clone._includes[field] = ManyToOneIncluder('__garbage__' + field.name, field)
            elif field.one_to_many:
                clone._includes[field] = OneToManyIncluder('__garbage__' + field.name, field)
        clone.__construct_extra()
        return clone
    
    def __construct_extra(self):
        for includer in self._includes.values():
            self.query.add_extra(includer.extra_query(), None, None, None, None, None)
            
    def _clone(self):
        clone = super(IncludeQuerySet, self)._clone()
        clone._includes = self._includes
        return clone
        
IncludeManager = IncludeQuerySet.as_manager()
IncludeManager.contribute_to_class(AbstractNode, 'bobjects')

In [ ]:
from django.db import models
from django.db.models.sql.query import Query
from django.db.models.sql.datastructures import Join
from django.db.models.query import ModelIterable
from django.contrib.contenttypes.models import ContentType
from django.contrib.contenttypes.fields import GenericRelation
from django.db.models.expressions import Func, Value, RawSQL
from django.db.models.expressions import OrderBy
from django.contrib.postgres.aggregates import ArrayAgg
from django.contrib.postgres.fields import JSONField
import ujson
import psycopg2
from collections import OrderedDict


def _load(s):
    if s == '{}':
        return {}
    return ujson.loads(s)
psycopg2.extras.register_default_json(globally=True, loads=_load)


class JSONBuildArray(Func):
    function = 'JSON_BUILD_ARRAY'
        
    def __init__(self, *args, **kwargs):
        super(JSONBuildArray, self).__init__(*args, output_field=JSONField(), **kwargs)
    

class JSONAgg(ArrayAgg):
    function = 'JSON_AGG'
    template = '%(function)s(%(expressions)s%(order_by)s)'
    
    def __init__(self, *args, **kwargs):
        super(JSONAgg, self).__init__(*args, output_field=JSONField(), **kwargs)
    
    def as_sql(self, compiler, connection, function=None, template=None):
        if self.extra.get('order_by'):
            self.extra['order_by'] = ' ORDER BY ' + ', '.join(compiler.compile(x)[0] for x in self.extra['order_by'])
        else:
            self.extra['order_by'] = ''
        return super(JSONAgg, self).as_sql(compiler, connection, function=None, template=None)
        

class IncludeModelIterable(ModelIterable):
    
    @classmethod
    def parse_nested(cls, instance, field, nested, datas):
        if field.many_to_one:
            datas = (datas, )
        ps = []    
        
        for data in datas or []:
            data, nested_data = data[:-len(nested) or None], data[-len(nested):]
            parsed = field.related_model.from_db(instance._state.db, None, data)
            for (f, n), d in zip(nested.items(), nested_data) :
                cls.parse_nested(parsed, f, n, d)
            ps.append(parsed)
            
        if field.many_to_one:
            return setattr(instance, field.get_cache_name(), ps[0])
        
        if not hasattr(instance, '_prefetched_objects_cache'):
            instance._prefetched_objects_cache = {}
        instance._prefetched_objects_cache[field.name] = field.related_model.objects.none()
        instance._prefetched_objects_cache[field.name]._result_cache = ps
        
    
    @classmethod
    def parse_includes(cls, instance, fields):
        for field, nested in fields.items():
            data = getattr(instance, '__' + field.name)
            delattr(instance, '__' + field.name)
            cls.parse_nested(instance, field, nested, data)
    
    def __iter__(self):
        for instance in super(IncludeModelIterable, self).__iter__():
            self.parse_includes(instance, self.queryset._includes)
                
            yield instance
            
            
class IncludeQuerySet(models.QuerySet):

    def __init__(self, *args, **kwargs):
        super(IncludeQuerySet, self).__init__(*args, **kwargs)
        self._include_limit = None
        self._includes = OrderedDict()
        self._iterable_class = IncludeModelIterable

    def include(self, *related_names, **kwargs):
        clone = self._clone()
        clone._include_limit = kwargs.pop('limit_includes', None)
        assert not kwargs, '"limit_includes" is the only accepted kwargs. Eat your heart out 2.7'
        
        for name in related_names:
            ctx, model = self._includes, self.model
            for spl in name.split('__'):
                field = model._meta.get_field(spl)
                model = field.related_model
                ctx = ctx.setdefault(field, OrderedDict())
                
        for field in self._includes.keys():
            clone._include(field)
            
        return clone
    
    def _include(self, field):
        sql, params = self._build_include_sql(field, self._includes[field])
        self.query.add_extra({'__' + field.name: sql}, params, None, None, None, None)        

    def _build_include_sql(self, field, children):
        host_model = field.model
        model = field.related_model
        
        if isinstance(field, GenericRelation):
            column, host_column = field.get_joining_columns()[0]
        else:
            host_column, column = field.get_joining_columns()[0]

        qs = model.objects.all()
        
        if self._include_limit:
            qs.query.set_limits(0, self._include_limit)
        
        kwargs = {}        
        if qs.ordered:
            kwargs['order_by'] = zip(*qs.query.get_compiler(using=self.db).get_order_by())[0]
        
        where = ['"{table}"."{column}" = "{host_table}"."{host_column}"'.format(
            table=model._meta.db_table,
            column=column,
            host_table=host_model._meta.db_table,
            host_column=host_column,
        )]
        
        if isinstance(field, GenericRelation):
            where.append('"{table}"."{content_type}" = {content_type_id}'.format(
                table=model._meta.db_table,
                content_type=model._meta.get_field(field.content_type_field_name).column,
                content_type_id=ContentType.objects.get_for_model(host_model).pk
            ))
 
        qs.query.add_extra(None, None, where, None, None, None)
    
        expressions = [f.column for f in model._meta.concrete_fields]
        
        for item in children.items():
            expressions.append(RawSQL(*self._build_include_sql(*item)))

        agg = JSONBuildArray(*expressions)

        if not field.many_to_one:
            agg = JSONAgg(agg, **kwargs)
        
        qs.query.add_annotation(agg, '__fields', is_summary=True)
        
        qs = qs.values_list('__fields')
        qs.query.clear_ordering(True)
        
        return qs.query.sql_with_params()

    def _clone(self):
        clone = super(IncludeQuerySet, self)._clone()
        clone._includes = self._includes
        return clone

    
IncludeQuerySet.as_manager().contribute_to_class(Guid, 'bobjects')
IncludeQuerySet.as_manager().contribute_to_class(AbstractNode, 'bobjects')
IncludeQuerySet.as_manager().contribute_to_class(Contributor, 'bobjects')
IncludeQuerySet.as_manager().contribute_to_class(OSFUser, 'bobjects')

In [ ]:
#print_sql(str(AbstractNode.bobjects.include('guids', 'contributor_set__user').query))

In [ ]:
%prun list(AbstractNode.bobjects.include('guids', 'contributor_set__user__guids')[:500])

In [ ]:
print(str(AbstractNode.bobjects.include('guids', 'contributor_set__user__guids', limit_includes=10).query))

In [ ]:
for x in AbstractNode.bobjects.include('guids', 'contributor_set__user__guids')[100:200]:
    print(x._id)
    print([contrib.user._id for contrib in x.contributor_set.all()])

In [ ]:
%pdb

In [ ]:
Guid._meta.get_field('id')

In [ ]:
Guid._meta.get_field('id').get_col('id').asc()

In [ ]:
print(str(AbstractNode.objects.all()[0].guids.all().order_by('id', 'created').query))

In [ ]:
zip(*AbstractNode.objects.all()[0].guids.all().order_by('id', 'created').query.get_compiler(using='default').get_order_by())[0]

In [ ]:
field = AbstractNode._meta.get_field('guids')

In [ ]:
AbstractNode.objects.all()[0].guids.all().query.get_compiler(using='default').get_order_by()

In [ ]:
from django.db.models.expressions import Func
from django.contrib.postgres.aggregates import ArrayAgg
from django.contrib.postgres.fields import JSONField


class JSONBuildArray(Func):
    function = 'json_build_array'

    
class JSONAgg(ArrayAgg):
    function = 'JSON_AGG'
    def __init__(self, *args):
        super(JSONAgg, self).__init__(*args, output_field=JSONField())


qs = Guid.objects.all().extra(where=['osf_guid.object_id = osf_abstractnode.id'])
qs.query.add_annotation(JSONAgg(JSONBuildArray('id', '_id')), 'ids', is_summary=True)
qs = qs.values_list('ids')
qs.query.order_by = []
qs.query.default_ordering = False
print(qs.query.__dict__)
sql, params = qs.query.sql_with_params()
print(sql)
AbstractNode.objects.all().extra(select={'_guids': sql}, select_params=params)

In [ ]:
print(str(AbstractNode.objects.select_related('root').query))

In [ ]:
AbstractNode.objects.filter(root__id=1).query.__dict__

In [ ]:
print(str(AbstractNode.objects.extra(tables=['osf_guid']).query))

In [ ]:
x = AbstractNode.objects.all().prefetch_related('tags').get(id=164233)

In [1]:
AbstractNode.objects.all().include('tags')

AttributeError: 'ManyToManyField' object has no attribute 'get_joining_columns'

In [11]:
print_sql(AbstractNode.objects.include('root__guids').query)

SELECT
  (SELECT JSON_BUILD_ARRAY("osf_abstractnode"."id", "osf_abstractnode"."guid_string", "osf_abstractnode"."content_type_pk", "osf_abstractnode"."type", "osf_abstractnode"."spam_status", "osf_abstractnode"."spam_pro_tip", "osf_abstractnode"."spam_data", "osf_abstractnode"."date_last_reported", "osf_abstractnode"."reports", "osf_abstractnode"."category", "osf_abstractnode"."child_node_subscriptions", "osf_abstractnode"."creator_id", "osf_abstractnode"."date_created", "osf_abstractnode"."date_modified", "osf_abstractnode"."deleted_date", "osf_abstractnode"."description", "osf_abstractnode"."file_guid_to_share_uuids", "osf_abstractnode"."forked_date", "osf_abstractnode"."forked_from_id", "osf_abstractnode"."is_fork", "osf_abstractnode"."is_public", "osf_abstractnode"."is_deleted", "osf_abstractnode"."node_license_id", "osf_abstractnode"."root_id", "osf_abstractnode"."piwik_site_id", "osf_abstractnode"."public_comments", "osf_abstractnode"."suspended", "osf_abstractnode"."template_nod

In [12]:
print_sql(AbstractNode.objects.select_related('root').query)

SELECT "osf_abstractnode"."id",
       "osf_abstractnode"."guid_string",
       "osf_abstractnode"."content_type_pk",
       "osf_abstractnode"."type",
       "osf_abstractnode"."spam_status",
       "osf_abstractnode"."spam_pro_tip",
       "osf_abstractnode"."spam_data",
       "osf_abstractnode"."date_last_reported",
       "osf_abstractnode"."reports",
       "osf_abstractnode"."category",
       "osf_abstractnode"."child_node_subscriptions",
       "osf_abstractnode"."creator_id",
       "osf_abstractnode"."date_created",
       "osf_abstractnode"."date_modified",
       "osf_abstractnode"."deleted_date",
       "osf_abstractnode"."description",
       "osf_abstractnode"."file_guid_to_share_uuids",
       "osf_abstractnode"."forked_date",
       "osf_abstractnode"."forked_from_id",
       "osf_abstractnode"."is_fork",
       "osf_abstractnode"."is_public",
       "osf_abstractnode"."is_deleted",
       "osf_abstractnode"."node_license_id",
       "osf_abstractnode"."root_id",
    

In [97]:
# q = AbstractNode.objects.select_related('root__root').query
q = AbstractNode.objects.select_related('root').query
q.__dict__

{'_annotation_select_cache': None,
 '_annotations': None,
 '_extra': None,
 '_extra_select_cache': None,
 'alias_map': {},
 'alias_refcount': {},
 'annotation_select_mask': None,
 'context': {},
 'default_cols': True,
 'default_ordering': True,
 'deferred_loading': (set(), True),
 'distinct': False,
 'distinct_fields': [],
 'external_aliases': set(),
 'extra_order_by': (),
 'extra_select_mask': None,
 'extra_tables': (),
 'filter_is_sticky': False,
 'group_by': None,
 'high_mark': None,
 'low_mark': 0,
 'max_depth': 5,
 'model': osf.models.node.AbstractNode,
 'order_by': [],
 'select': [],
 'select_for_update': False,
 'select_for_update_nowait': False,
 'select_related': {'root': {}},
 'standard_ordering': True,
 'table_map': {},
 'tables': [],
 'used_aliases': set(),
 'values_select': [],
 'where': <WhereNode: (AND: )>,
 'where_class': django.db.models.sql.where.WhereNode}

In [89]:
%prun q.get_compiler('default').get_select()

In [93]:
# q.get_compiler('default').get_select()

In [94]:
# q.table_alias('test_', create=True)
# self.table_alias(join.table_name, create=True)
# q.bump_prefix(AbstractNode.objects.all().query)

In [100]:
q.__dict__

{'_annotation_select_cache': None,
 '_annotations': None,
 '_extra': None,
 '_extra_select_cache': None,
 'alias_map': {u'osf_abstractnode': <django.db.models.sql.datastructures.BaseTable at 0x117274cd0>},
 'alias_refcount': {'T2': 1, u'osf_abstractnode': 1},
 'annotation_select_mask': None,
 'context': {},
 'default_cols': True,
 'default_ordering': True,
 'deferred_loading': (set(), True),
 'distinct': False,
 'distinct_fields': [],
 'external_aliases': set(),
 'extra_order_by': (),
 'extra_select_mask': None,
 'extra_tables': (),
 'filter_is_sticky': False,
 'group_by': None,
 'high_mark': None,
 'low_mark': 0,
 'max_depth': 5,
 'model': osf.models.node.AbstractNode,
 'order_by': [],
 'select': [],
 'select_for_update': False,
 'select_for_update_nowait': False,
 'select_related': {'root': {}},
 'standard_ordering': True,
 'table_map': {u'osf_abstractnode': [u'osf_abstractnode', 'T2']},
 'tables': [u'osf_abstractnode', 'T2'],
 'used_aliases': set(),
 'values_select': [],
 'where': <

In [98]:
q.get_initial_alias()

u'osf_abstractnode'

In [99]:
q.table_alias('osf_abstractnode', True)

('T2', True)

In [180]:
from django.db.models.sql.datastructures import (
    BaseTable, Empty, EmptyResultSet, Join, MultiJoin,
)

q1 = AbstractNode.objects.all().select_related('root').query
q2 = AbstractNode.objects.all().select_related('root').query

bt = BaseTable('osf_abstractnode', None)

q1.sql_with_params()

print(q1.get_initial_alias())

q2.table_map = q1.table_map.copy()
q2.alias_map = q1.alias_map.copy()

print(q2.get_initial_alias())

osf_abstractnode
T3


In [181]:
print_sql(q2)

SELECT T3."id",
       T3."guid_string",
       T3."content_type_pk",
       T3."type",
       T3."spam_status",
       T3."spam_pro_tip",
       T3."spam_data",
       T3."date_last_reported",
       T3."reports",
       T3."category",
       T3."child_node_subscriptions",
       T3."creator_id",
       T3."date_created",
       T3."date_modified",
       T3."deleted_date",
       T3."description",
       T3."file_guid_to_share_uuids",
       T3."forked_date",
       T3."forked_from_id",
       T3."is_fork",
       T3."is_public",
       T3."is_deleted",
       T3."node_license_id",
       T3."root_id",
       T3."piwik_site_id",
       T3."public_comments",
       T3."suspended",
       T3."template_node_id",
       T3."title",
       T3."wiki_pages_current",
       T3."wiki_pages_versions",
       T3."wiki_private_uuids",
       T3."preprint_file_id",
       T3."preprint_article_doi",
       T3."_is_preprint_orphan",
       T3."_has_abandoned_preprint",
       T3."keenio_read_key",


In [177]:
q.__dict__

{'_annotation_select_cache': None,
 '_annotations': None,
 '_extra': None,
 '_extra_select_cache': None,
 'alias_map': {u'osf_abstractnode': <django.db.models.sql.datastructures.BaseTable at 0x117274cd0>},
 'alias_refcount': {'T2': 1, u'osf_abstractnode': 1},
 'annotation_select_mask': None,
 'context': {},
 'default_cols': True,
 'default_ordering': True,
 'deferred_loading': (set(), True),
 'distinct': False,
 'distinct_fields': [],
 'external_aliases': set(),
 'extra_order_by': (),
 'extra_select_mask': None,
 'extra_tables': (),
 'filter_is_sticky': False,
 'group_by': None,
 'high_mark': None,
 'low_mark': 0,
 'max_depth': 5,
 'model': osf.models.node.AbstractNode,
 'order_by': [],
 'select': [],
 'select_for_update': False,
 'select_for_update_nowait': False,
 'select_related': {'root': {}},
 'standard_ordering': True,
 'table_map': {u'osf_abstractnode': [u'osf_abstractnode',
   'T2',
   'T2',
   'T3',
   'T2',
   'T3',
   'T2',
   'T3']},
 'tables': [u'osf_abstractnode', 'T2'],
